<a href="https://www.kaggle.com/code/vincemarcs/mvsa-image-models?scriptVersionId=101480617" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
SEED = 61

import os
import re
import gc
import h5py
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA
from IPython.display import display_html
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation, RepeatVector, Permute
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPool2D, Convolution2D
from keras.layers.core import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [2]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')

    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [3]:
def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]

    return labels, text_labels, image_labels

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def load_mvsa_images(merge=False):
    folder_path = '../input/mvsa-data'
    file_paths = os.listdir(folder_path)
    for path in file_paths:
        file_name = os.path.split(path)[1]
        if file_name.split('.')[1] == 'npz':
            if file_name.split('-')[1] == 'single':
                mvsa_single_images_path = os.path.join(folder_path, path)
            else:
                mvsa_multiple_images_path = os.path.join(folder_path, path)
    
    mvsa_single = loadz(mvsa_single_images_path)
    mvsa_multiple = loadz(mvsa_multiple_images_path)
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

In [4]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 4, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 3)
    plt.plot(history.history['f1_macro'])
    plt.plot(history.history['val_f1_macro'])
    plt.title('Macro F1-SCORE')
    plt.ylabel('f1-macro')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')
    
    fig.add_subplot(1, 4, 4)
    plt.plot(history.history['f1_weighted'])
    plt.plot(history.history['val_f1_weighted'])
    plt.title('Weighted F1-SCORE')
    plt.ylabel('f1-weighted')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [5]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * len(data))
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_tf_data(data, validation_split):
    DATASET_SIZE = len(data)
    train_size = int((1-validation_split*2) * DATASET_SIZE)
    val_size = int(validation_split * DATASET_SIZE)
    test_size = int(validation_split * DATASET_SIZE)

#     full_dataset = tf.data.TFRecordDataset(FLAGS.input_file)
#     full_dataset = full_dataset.shuffle()
    train_dataset = data.take(train_size)
    test_dataset = data.skip(train_size)
    val_dataset = test_dataset.skip(test_size)
    test_dataset = test_dataset.take(test_size)
    return train_dataset, val_dataset, test_dataset

In [6]:
NUM_CLASSES = 3
f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')

#     # soft attention
#     attention = Dense(1, activation='tanh') (image_input)
#     attention = Flatten() (attention)
#     attention = Activation('softmax') (attention)
#     attention = RepeatVector(NUM_HIDDEN) (attention)
#     attention = Permute([2, 1]) (attention)
#     attention = Flatten() (attention)

def create_model_pretrained(input_shape, lstm=True):
    image_input = Input(shape=input_shape)
    dropout = Dropout(DROPOUT_INPUT_IMG) (image_input)    
    if lstm == True:
        image_reshape = Reshape((1, -1)) (dropout)
        image_lstm = LSTM(NUM_LSTM_IMG) (image_reshape)
        dropout = Dropout(DROPOUT_LSTM_IMG) (image_lstm)
    outputs = Dense(NUM_CLASSES, activation='softmax') (dropout)
    model = Model(image_input, outputs)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy', f1_macro, f1_weighted])
    return model

2022-07-22 10:42:22.938229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 10:42:22.939307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 10:42:22.939964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 10:42:22.940797: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))
    
    loss, acc, f1_macro, f1_weighted = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('Macro F1-score:', f1_macro)
        print('Weighted F1-score:', f1_weighted)
        
    return loss, acc, f1_macro, f1_weighted

In [8]:
def run_and_evaluate(name, X, y, verbose=0, lstm=False):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    if 'multiple' in name:
        batch_size = BATCH_SIZE_MULTIPLE
        if DO_SMOTE_MULTIPLE == True:
#             oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
            oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
            X_train, y_train = oversample.fit_resample(X_train, y_train)
    else:
        batch_size = BATCH_SIZE_SINGLE # 128
        if DO_SMOTE_SINGLE == True:
#             oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
            oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
            X_train, y_train = oversample.fit_resample(X_train, y_train)
        
    model = create_model_pretrained(X_train.shape[1:], lstm=lstm)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
#     early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=PATIENCE)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=PATIENCE)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                                   epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                                   callbacks=[checkpoint, reduce_lr])
    best_epoch = np.argmin(history.history['val_loss'])
    print('Checkpoint loaded at epoch:', best_epoch)

    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [9]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:tomato', axis=0)\
                          .highlight_min(subset=['Loss'], props='color:lawngreen', axis=0)\
                          .highlight_max(subset=['Loss'], props='color:tomato', axis=0)

def style_dataframe_out(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-weighted'], props='color:tomato', axis=0)
                          
def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)
    return html_str

# Load data

In [10]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, mvsa_single_image_labels = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, mvsa_multiple_image_labels = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3

In [11]:
# prepare all features data
feature_names = ['xception', 'vgg16', 'vgg19', 'resnet50', 'resnet101', 'resnet152', 'densenet121', 'densenet169', 'densenet201']

mvsa_single_features = []
mvsa_multiple_features = []
mvsa_features = []

for name in tqdm(feature_names):
    data = load_mvsa_feature(name)
    merge_data = merge_mvsa(data[0], data[1])

    mvsa_single_features.append(data[0])
    mvsa_multiple_features.append(data[1])
    mvsa_features.append(merge_data)

100%|██████████| 9/9 [00:14<00:00,  1.60s/it]


In [12]:
def shuffle_mvsa(mvsa_features, labels, indices):
    shuffled_features = []
#     random_idx = np.random.permutation(len(labels))
    for i in range(len(mvsa_features)):
        x = mvsa_features[i][indices]
        shuffled_features.append(x)
    return shuffled_features, labels[indices]

In [13]:
# Fix random indices for consistency between other experiments
mvsa_single_features, mvsa_single_multimodal_labels = shuffle_mvsa(mvsa_single_features, mvsa_single_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-single-shuffle-indices.npy'))
mvsa_multiple_features, mvsa_multiple_multimodal_labels = shuffle_mvsa(mvsa_multiple_features, mvsa_multiple_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-multiple-shuffle-indices.npy'))

# Run models and Evalution display

In [14]:
reset_seeds()
EPOCHS = 100
VALIDATION_SPLIT = 0.1
PATIENCE = 10

BATCH_SIZE_SINGLE = 128
BATCH_SIZE_MULTIPLE = 256

DO_SMOTE_SINGLE = False
DO_SMOTE_MULTIPLE = False

HAS_LSTM = False
# NUM_LSTM_IMG = 64
DROPOUT_INPUT_IMG = 0.0
DROPOUT_LSTM_IMG = 0.0

OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'

## With original image labels

In [15]:
# print('MVSA-Single: With original image labels')
# mvsa_single_histories = []
# mvsa_single_scores = []
# for i in range(len(feature_names)):        
#     print('MVSA-Single:', feature_names[i])
#     if feature_names[i] == 'cnn':
#         history, score = run_and_evaluate_cnn('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_image_labels, verbose=0)
#     else:
#         history, score = run_and_evaluate('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_image_labels, verbose=0)
#     mvsa_single_histories.append(history)
#     mvsa_single_scores.append(score)
#     print()
# df_single_scores = pd.DataFrame(mvsa_single_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# print('MVSA-Multiple: With original image labels')
# mvsa_multiple_histories = []
# mvsa_multiple_scores = []
# for i in range(len(feature_names)):
# #     print('MVSA-Multiple:', feature_names[i])
#     if feature_names[i] == 'cnn':
#         history, score = run_and_evaluate_cnn('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_image_labels, verbose=1)
#     else:
#         history, score = run_and_evaluate('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_image_labels, verbose=0)
#     mvsa_multiple_histories.append(history)
#     mvsa_multiple_scores.append(score)
#     print()
# df_multiple_scores = pd.DataFrame(mvsa_multiple_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# mvsa_average_scores = np.mean([mvsa_single_scores, mvsa_multiple_scores], axis=0)
# df_average_scores = pd.DataFrame(mvsa_average_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

## With multimodal labels

In [16]:
print('MVSA-Single: With multimodal labels')
mvsa_single_histories2 = []
mvsa_single_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    history, score = run_and_evaluate('single-ML-' + feature_names[i], mvsa_single_features[i], mvsa_single_multimodal_labels, 
                                      verbose=0, lstm=HAS_LSTM)
    mvsa_single_histories2.append(history)
    mvsa_single_scores2.append(score)
    print()
df_single_scores2 = pd.DataFrame(mvsa_single_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('MVSA-Multiple: With multimodal labels')
mvsa_multiple_histories2 = []
mvsa_multiple_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    history, score = run_and_evaluate('multiple-ML-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_multimodal_labels,
                                      verbose=0, lstm=HAS_LSTM)
    mvsa_multiple_histories2.append(history)
    mvsa_multiple_scores2.append(score)
    print()
df_multiple_scores2 = pd.DataFrame(mvsa_multiple_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

mvsa_average_scores2 = np.mean([mvsa_single_scores2, mvsa_multiple_scores2], axis=0)
df_average_scores2 = pd.DataFrame(mvsa_average_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single: With multimodal labels
MVSA-Single: xception


2022-07-22 10:42:43.852225: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Checkpoint loaded at epoch: 9

MVSA-Single: vgg16
Checkpoint loaded at epoch: 12

MVSA-Single: vgg19
Checkpoint loaded at epoch: 12

MVSA-Single: resnet50
Checkpoint loaded at epoch: 31

MVSA-Single: resnet101
Checkpoint loaded at epoch: 23

MVSA-Single: resnet152
Checkpoint loaded at epoch: 31

MVSA-Single: densenet121
Checkpoint loaded at epoch: 16

MVSA-Single: densenet169
Checkpoint loaded at epoch: 5

MVSA-Single: densenet201
Checkpoint loaded at epoch: 10

MVSA-Multiple: With multimodal labels
MVSA-Multiple: xception
Checkpoint loaded at epoch: 8

MVSA-Multiple: vgg16
Checkpoint loaded at epoch: 34

MVSA-Multiple: vgg19
Checkpoint loaded at epoch: 25

MVSA-Multiple: resnet50
Checkpoint loaded at epoch: 23

MVSA-Multiple: resnet101
Checkpoint loaded at epoch: 34

MVSA-Multiple: resnet152
Checkpoint loaded at epoch: 40

MVSA-Multiple: densenet121
Checkpoint loaded at epoch: 8

MVSA-Multiple: densenet169
Checkpoint loaded at epoch: 11

MVSA-Multiple: densenet201
Checkpoint loaded at

## With merge MVSA data

In [17]:
# print('Both MVSA: With original image labels')
# mvsa_histories3 = []
# mvsa_scores3 = []
# for i in range(len(feature_names)):
#     print('Both MVSA:', feature_names[i])
#     history, score = run_and_evaluate('merge-OL-' + feature_names[i], mvsa_features[i], mvsa_image_labels, verbose=0)
#     mvsa_histories3.append(history)
#     mvsa_scores3.append(score)
#     print()
# df_scores3 = pd.DataFrame(mvsa_scores3, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# print('Both MVSA: With multimodal labels')
# mvsa_histories4 = []
# mvsa_scores4 = []
# for i in range(len(feature_names)):
#     print('Both MVSA:', feature_names[i])
#     history, score = run_and_evaluate('merge-ML-' + feature_names[i], mvsa_features[i], mvsa_multimodal_labels, verbose=0)
#     mvsa_histories4.append(history)
#     mvsa_scores4.append(score)
#     print()
# df_scores4 = pd.DataFrame(mvsa_scores4, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# Display results

In [18]:
# print('With original image labels\n')
# display_dataframes((style_dataframe(df_single_scores), style_dataframe(df_multiple_scores), style_dataframe(df_average_scores)), 
#                    names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

In [19]:
print('With multimodal labels\n')
display_dataframes((style_dataframe(df_single_scores2), style_dataframe(df_multiple_scores2), style_dataframe(df_average_scores2)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])
print()

With multimodal labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.777739 
 0.658537 
 0.503470 
 0.633624 
 
 
 vgg16 
 0.814245 
 0.636364 
 0.469070 
 0.604101 
 
 
 vgg19 
 0.816350 
 0.634146 
 0.454185 
 0.595032 
 
 
 resnet50 
 0.895692 
 0.592018 
 0.276286 
 0.466380 
 
 
 resnet101 
 0.888614 
 0.580931 
 0.269353 
 0.458057 
 
 
 resnet152 
 0.886435 
 0.580931 
 0.261288 
 0.449893 
 
 
 densenet121 
 0.823633 
 0.596452 
 0.447714 
 0.578489 
 
 
 densenet169 
 0.790157 
 0.643016 
 0.522780 
 0.631867 
 
 
 densenet201 
 0.792787 
 0.614191 
 0.478295 
 0.607113 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.770560 
 0.656874 
 0.329000 
 0.570186 
 
 
 vgg16 
 0.776004 
 0.663925 
 0.317738 
 0.562941 
 
 
 vgg19 
 0.784184 
 0.664512 
 0.320475 
 0.561918 
 
 
 resnet50 
 0.801675 
 0.660400 
 0.286790 
 0.543063 
 
 
 resnet101 
 0.807653 
 0.659812 
 0.282776 
 0.539812 
 
 
 resnet152 
 0.808647 
 0.660987 
 0.294173 
 0.543296 
 
 
 densenet121 
 0.785842 
 0.647474 
 0.323348 
 0.565662 
 
 
 densenet169 
 0.784796 
 0.657462 
 0.338375 
 0.576566 
 
 
 densenet201 
 0.790055 
 0.660987 
 0.367406 
 0.593486 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.774149 
 0.657705 
 0.416235 
 0.601905 
 
 
 vgg16 
 0.795125 
 0.650144 
 0.393404 
 0.583521 
 
 
 vgg19 
 0.800267 
 0.649329 
 0.387330 
 0.578475 
 
 
 resnet50 
 0.848684 
 0.626209 
 0.281538 
 0.504721 
 
 
 resnet101 
 0.848134 
 0.620372 
 0.276065 
 0.498935 
 
 
 resnet152 
 0.847541 
 0.620959 
 0.277730 
 0.496595 
 
 
 densenet121 
 0.804737 
 0.621963 
 0.385531 
 0.572075 
 
 
 densenet169 
 0.787477 
 0.650239 
 0.430578 
 0.604217 
 
 
 densenet201 
 0.791421 
 0.637589 
 0.422851 
 0.600299

In [20]:
drop_columns = ['Loss', 'F1-macro']
def dataframe_to_display(df):
    return style_dataframe_out(np.round(df, 3).drop(columns=drop_columns))

In [21]:
if not os.path.exists('./tables'):
    os.makedirs('./tables')
open('./tables/single_image_scores.html', 'w').write(dataframe_to_display(df_single_scores2).to_html())
open('./tables/multiple_image_scores.html', 'w').write(dataframe_to_display(df_multiple_scores2).to_html())
open('./tables/average_image_scores.html', 'w').write(dataframe_to_display(df_average_scores2).to_html())

2601

In [22]:
# print('With both MVSA merged together\n')
# display_dataframes((style_dataframe(df_scores3), style_dataframe(df_scores4)), 
#                    names=['Original image labels', 'Multimodal labels'])

# Plots

In [23]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
WIDTH = 650
HEIGHT = 400
if not os.path.exists('./plots'):
    os.makedirs('./plots')

In [24]:
df_single = pd.concat([df_single_scores2['Accuracy'], df_single_scores2['F1-weighted']], axis=1)
df_single['Đặc trưng'] = df_single.index
df_single['Bộ dữ liệu'] = 'MVSA-Single'
df_multiple = pd.concat([df_multiple_scores2['Accuracy'], df_multiple_scores2['F1-weighted']], axis=1)
df_multiple['Đặc trưng'] = df_multiple.index
df_multiple['Bộ dữ liệu'] = 'MVSA-Multiple'
df = pd.concat([df_single, df_multiple])

In [25]:
import plotly.express as px

fig = px.bar(df, x="Đặc trưng", y=["Accuracy", "F1-weighted"], facet_col="Bộ dữ liệu", facet_col_spacing = 0.15)
fig.update_layout(barmode = 'group')
# fig.update_layout(bargap=0.2)
fig.update_layout(margin_b=25, margin_t=50, margin_l=25, margin_r=25, width=WIDTH, height=HEIGHT)
fig.update_layout(title='Biểu đồ kết quả đánh giá các đặc trưng của dữ liệu hình ảnh', 
#               xaxis_title='Đặc trưng', 
              yaxis_title='Điểm', legend_title='Độ đo')
fig.write_html('./plots/image-scores-plot.html')
fig.show()

# Dratfs

In [26]:
# # load separate
# mvsa_single_images, mvsa_multiple_images = load_mvsa_images()
# mvsa_single_xception, mvsa_multiple_xception = load_mvsa_feature('xception')
# mvsa_single_vgg16, mvsa_multiple_vgg16 = load_mvsa_feature('vgg16')
# mvsa_single_vgg19, mvsa_multiple_vgg19 = load_mvsa_feature('vgg19')
# mvsa_single_resnet50, mvsa_multiple_resnet50 = load_mvsa_feature('resnet50')
# mvsa_single_resnet101, mvsa_multiple_resnet101 = load_mvsa_feature('resnet101')
# mvsa_single_resnet152, mvsa_multiple_resnet152 = load_mvsa_feature('resnet152')
# mvsa_single_densenet121, mvsa_multiple_densenet121 = load_mvsa_feature('densenet121')
# mvsa_single_densenet169, mvsa_multiple_densenet169 = load_mvsa_feature('densenet169')
# mvsa_single_densenet201, mvsa_multiple_densenet201 = load_mvsa_feature('densenet201')

# # load merge
# mvsa_images = merge_mvsa(mvsa_single_images, mvsa_multiple_images)
# mvsa_xception = merge_mvsa(mvsa_single_xception, mvsa_multiple_xception)
# mvsa_vgg16 = merge_mvsa(mvsa_single_vgg16, mvsa_multiple_vgg16)
# mvsa_vgg19 = merge_mvsa(mvsa_single_vgg19, mvsa_multiple_vgg19)
# mvsa_resnet50 = merge_mvsa(mvsa_single_resnet50, mvsa_multiple_resnet50)
# mvsa_resnet101 = merge_mvsa(mvsa_single_resnet101, mvsa_multiple_resnet101)
# mvsa_resnet152 = merge_mvsa(mvsa_single_resnet152, mvsa_multiple_resnet152)
# mvsa_densenet121 = merge_mvsa(mvsa_single_densenet121, mvsa_multiple_densenet121)
# mvsa_densenet169 = merge_mvsa(mvsa_single_densenet169, mvsa_multiple_densenet169)
# mvsa_densenet201 = merge_mvsa(mvsa_single_densenet201, mvsa_multiple_densenet201)

# # prepare all features data
# feature_names = ['cnn', 'xception', 'vgg16', 'vgg19', 'resnet50', 'resnet101', 'resnet152', 'densenet121', 'densenet169', 'densenet201']

# mvsa_single_features = [mvsa_single_images,
#                         mvsa_single_xception,
#                         mvsa_single_vgg16, mvsa_single_vgg19, 
#                         mvsa_single_resnet50, mvsa_single_resnet101, mvsa_single_resnet152, 
#                         mvsa_single_densenet121, mvsa_single_densenet169, mvsa_single_densenet201]

# mvsa_multiple_features = [mvsa_multiple_images,
#                           mvsa_multiple_xception,
#                           mvsa_multiple_vgg16, mvsa_multiple_vgg19, 
#                           mvsa_multiple_resnet50, mvsa_multiple_resnet101, mvsa_multiple_resnet152, 
#                           mvsa_multiple_densenet121, mvsa_multiple_densenet169, mvsa_multiple_densenet201]

# mvsa_features = [mvsa_images,
#                  mvsa_xception,
#                  mvsa_vgg16, mvsa_vgg19, 
#                  mvsa_resnet50, mvsa_resnet101, mvsa_resnet152, 
#                  mvsa_densenet121, mvsa_densenet169, mvsa_densenet201]